<h3> ML Pipeline Preparation</h3><br>
### 1. Import libraries and load data from database.<br>
- Import Python libraries<br>
- Load dataset from database references(https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)?<br>
- Define feature and target variables X and Y

In [5]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3
import sqlalchemy
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


nltk.download(['punkt', 'wordnet'])

[nltk_data] Downloading package punkt to /home/alex/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
# load data from database
engine = create_engine('sqlite:///disastermessages.db')
df = pd.read_sql_table("disaster_messages", con=engine)

In [7]:
df.head()

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
#Create features 

X = df['message']
Y = df.iloc[:, 4:]

In [31]:
Y.head(1)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Tokenize

In [10]:
def tokenize(text):
    
    """
    -load data to word_tokenize library.
    - lematize the text
    """
    
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens=[]
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
        
    return clean_tokens

Build a machine learning pipeline

- `message` is the input column
- output is the other 36 columns


In [11]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

<h3> Training </h3> 

Plan
- Split data into train and test sets
- Train pipeline

In [12]:
#Split data using train_test_split funcition

X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [13]:
#fit the transformer
pipeline.fit(X_train, y_train)

/home/alex/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f534a415e10>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

<h4> Test your model </h4> 
Plan

- Report the f1 score, precision and recall for each output category of the dataset. 
- Iterate through the columns and calling sklearn's `classification_report` on each.

In [14]:
y_pred = pipeline.predict(X_test)

In [15]:
def test_model(y_test, y_pred):
    
    """
    Function to iterate through columns and call sklearn classification report on each.
    """
    for index, column in enumerate(y_test):
        print(column, classification_report(y_test[column], y_pred[:, index]))

In [16]:
test_model(y_test, y_pred)

related               precision    recall  f1-score   support

           0       0.73      0.28      0.40      1500
           1       0.82      0.97      0.89      5054

    accuracy                           0.81      6554
   macro avg       0.78      0.62      0.65      6554
weighted avg       0.80      0.81      0.78      6554

request               precision    recall  f1-score   support

           0       0.89      0.99      0.94      5453
           1       0.88      0.42      0.57      1101

    accuracy                           0.89      6554
   macro avg       0.89      0.70      0.75      6554
weighted avg       0.89      0.89      0.88      6554

offer               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      655

/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packa

water               precision    recall  f1-score   support

           0       0.95      1.00      0.97      6132
           1       0.90      0.23      0.37       422

    accuracy                           0.95      6554
   macro avg       0.92      0.62      0.67      6554
weighted avg       0.95      0.95      0.93      6554

food               precision    recall  f1-score   support

           0       0.92      1.00      0.96      5782
           1       0.91      0.37      0.52       772

    accuracy                           0.92      6554
   macro avg       0.92      0.68      0.74      6554
weighted avg       0.92      0.92      0.91      6554

shelter               precision    recall  f1-score   support

           0       0.93      1.00      0.96      5993
           1       0.84      0.26      0.40       561

    accuracy                           0.93      6554
   macro avg       0.89      0.63      0.68      6554
weighted avg       0.93      0.93      0.92      6554



/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packa

aid_centers               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6462
           1       0.00      0.00      0.00        92

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.97      0.99      0.98      6554

other_infrastructure               precision    recall  f1-score   support

           0       0.95      1.00      0.98      6255
           1       0.00      0.00      0.00       299

    accuracy                           0.95      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.91      0.95      0.93      6554

weather_related               precision    recall  f1-score   support

           0       0.86      0.96      0.91      4746
           1       0.86      0.60      0.71      1808

    accuracy                           0.86      6554
   macro avg       0.86      0.78      0.81      6554
weighted avg       0.86  

/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


<h4> Improvements </h4>

Plan

- Use grid search to find better parameters. 

In [17]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f534a415e10>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7f534a415e10>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf_

In [18]:
# specify parameters for grid search 
parameters = {
    'clf__estimator__n_estimators' : [50, 100]
}

In [19]:
# create grid search object 
cv = GridSearchCV(pipeline, param_grid=parameters)

cv

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f534a415e10>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 100]})

In [20]:
cv.fit(X_train, y_train)

/home/alex/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/alex/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/alex/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/alex/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/alex/.local/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/home/alex/.local/lib/pyt

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f534a415e10>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__n_estimators': [50, 100]})

In [21]:
cv.best_params_

{'clf__estimator__n_estimators': 100}

<h4> Retest the model </h4> 

- Show the accuracy, precision, and recall of the tuned model.  

In [22]:
y_pred = cv.predict(X_test)

In [29]:


test_model(y_test, y_pred)


related               precision    recall  f1-score   support

           0       0.73      0.28      0.41      1500
           1       0.82      0.97      0.89      5054

    accuracy                           0.81      6554
   macro avg       0.78      0.63      0.65      6554
weighted avg       0.80      0.81      0.78      6554

request               precision    recall  f1-score   support

           0       0.89      0.99      0.94      5453
           1       0.87      0.41      0.56      1101

    accuracy                           0.89      6554
   macro avg       0.88      0.70      0.75      6554
weighted avg       0.89      0.89      0.87      6554

offer               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6525
           1       0.00      0.00      0.00        29

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      655

/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packa

child_alone               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6554

    accuracy                           1.00      6554
   macro avg       1.00      1.00      1.00      6554
weighted avg       1.00      1.00      1.00      6554

water               precision    recall  f1-score   support

           0       0.95      1.00      0.97      6132
           1       0.91      0.21      0.35       422

    accuracy                           0.95      6554
   macro avg       0.93      0.61      0.66      6554
weighted avg       0.95      0.95      0.93      6554

food               precision    recall  f1-score   support

           0       0.92      0.99      0.96      5782
           1       0.88      0.39      0.54       772

    accuracy                           0.92      6554
   macro avg       0.90      0.69      0.75      6554
weighted avg       0.92      0.92      0.91      6554

shelter               precision    recall  f1-scor

/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packa

other_aid               precision    recall  f1-score   support

           0       0.87      1.00      0.93      5694
           1       0.69      0.01      0.02       860

    accuracy                           0.87      6554
   macro avg       0.78      0.50      0.48      6554
weighted avg       0.85      0.87      0.81      6554

infrastructure_related               precision    recall  f1-score   support

           0       0.93      1.00      0.96      6096
           1       0.00      0.00      0.00       458

    accuracy                           0.93      6554
   macro avg       0.47      0.50      0.48      6554
weighted avg       0.87      0.93      0.90      6554

transport               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6256
           1       0.78      0.07      0.13       298

    accuracy                           0.96      6554
   macro avg       0.87      0.53      0.55      6554
weighted avg       0.95      0.

/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/alex/.local/lib/python3.10/site-packa

floods               precision    recall  f1-score   support

           0       0.95      1.00      0.97      6024
           1       0.90      0.38      0.54       530

    accuracy                           0.95      6554
   macro avg       0.92      0.69      0.75      6554
weighted avg       0.94      0.95      0.94      6554

storm               precision    recall  f1-score   support

           0       0.94      0.99      0.96      5952
           1       0.74      0.40      0.52       602

    accuracy                           0.93      6554
   macro avg       0.84      0.69      0.74      6554
weighted avg       0.92      0.93      0.92      6554

fire               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6481
           1       0.00      0.00      0.00        73

    accuracy                           0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

e

In [30]:
accuracy = (y_pred == y_test).mean()
accuracy

related                   0.812328
request                   0.891059
offer                     0.995575
aid_related               0.766097
medical_help              0.924168
medical_products          0.946750
search_and_rescue         0.974062
security                  0.979097
military                  0.966433
child_alone               1.000000
water                     0.947971
food                      0.921422
shelter                   0.932560
clothing                  0.981996
money                     0.979402
missing_people            0.987489
refugees                  0.964754
death                     0.957125
other_aid                 0.869545
infrastructure_related    0.929966
transport                 0.956820
buildings                 0.952701
electricity               0.976198
tools                     0.992218
hospitals                 0.988099
shops                     0.995728
aid_centers               0.985963
other_infrastructure      0.954379
weather_related     


<code> More featurres or trying other ML algorithms can help in improving the ml model </code>

<h4> Export the model as pickel file </h4>

In [31]:
import pickle
filename = 'model.pkl'
pickle.dump(cv, open(filename, 'wb'))